# Plots
Notebook to plot ROC curves and confusion matrices for single models and ensemble models, for the different tasks under study (anomaly detection, vessel classification and course classification). The plots can be generated for the entire test sets or stratified by gender.

The notebook uses the output files named `predictions_test_*.xlsx` that contain predictions for single models and ensemble models, which are generated by the script `plot.py`.

In [ ]:
import os
from os.path import join
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pandas as pd
import numpy as np
from roc_utils import plot_roc, plot_roc_bootstrap, compute_roc, plot_mean_roc
from tableone import TableOne

from evaluation_core import plot_confusion_matrix, add_gender

titles_dict = { 
    "train": f"Train",
    "test_internal": f"Internal Test",
    "test_external": f"External Test"
}
sns.set_theme(style="white")
color_palette = matplotlib.colormaps.get_cmap('tab10').colors

In [ ]:
gender = "" # "m" "f" ""

n_boot = 1000  # number of bootstrap iterations for ROC confidence intervals
threshold_title = "Cut-Off"
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  # Other thresholds explored
threshold = 0.5

dpi = 300
input_path = "raw_data_plots_tables"
output_folder = "images/paper/"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
tasks = [
    "anomaly_detection",
    "origin_classification",
    "risk_classification",
]

In [ ]:
show = False
def plt_show():
    if show:
        plt.show()
    else:
        plt.close()

## Figure 3

In [ ]:
strategy = "train"
for task in tasks:
    df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
    df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))
    
    for df, set_name, color in zip([df_test_internal, df_test_external], ["test_internal", "test_external"], ["red", "blue"]):
        # PLot ROC curves for single models
        roc_curves = []
        for i in [0, 1, 2, 3, 4]:
            roc = compute_roc(X=df[f"model_{i}"], y=df["label"], pos_label=1)
            roc_curves.append(roc)
            plot_roc(roc, label=f"Model {i + 1}", color=color_palette[i])
        plt.title(add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - 5-Folds", gender, True))
        plt.savefig(f"images/paper/Figure3_{task}_{set_name}_singleModels.png", dpi=dpi)
        plt_show()
        
        # Plot ROC curve for the ensemble model
        plot_roc_bootstrap(X=df["ensemble"], y=df["label"], pos_label=1, label=f"{titles_dict[set_name]}",
                       color=color, n_bootstrap=n_boot, show_boots=False)
        plt.title(add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - Ensemble", gender, True)) 
        plt.suptitle("")     # Clear the subtitle
        plt.savefig(f"images/paper/Figure3_{task}_{set_name}_ensemble.png", dpi=dpi)
        plt_show()

## Figure 4

In [ ]:
strategy = "train"
for task in tasks:
    df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
    df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))
    
    for df, set_name in zip([df_test_internal, df_test_external], ["test_internal", "test_external"]):
        title = add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]}", gender, True)
        plot_confusion_matrix(predictions=df["ensemble"] > threshold, labels=df["label"].astype(int),
                              output_path=f"images/paper/Figure4_{task}_{set_name}_ensemble.png",
                              title=title, show=show, dpi=dpi)
        plt_show()

## Figure 6

In [ ]:
data_input_path = "latent_features"
base_colors = [r"#ff7979", r"#50ad76", r"#d36efa"]
colors = base_colors[:2] * 3
markers = ["^", "^", "o", "o", "X", "X"] 
task = "anomaly_detection"
size = 20

combination = ('test_external', 'train', 'test_internal')
loc = "upper right"
path_tsne = join(input_path, "tsne_" + "-".join(combination) + ".xlsx")
df_tsne = pd.read_excel(path_tsne)
df_tsne = df_tsne.sort_values("labels_anomaly")

plt.figure(figsize=(16, 16))
sns.scatterplot(x='tsne_1', y='tsne_2', 
                hue='labels_anomaly', 
                style="labels_anomaly",
                data=df_tsne, 
                s=50, 
                palette=colors[:2 * len(combination)],
                markers=markers[:2 * len(combination)]
               )
plt.xlim((np.min(df_tsne["tsne_1"]) - 1, np.max(df_tsne["tsne_1"]) + 1))
plt.ylim((np.min(df_tsne["tsne_2"]) - 1, np.max(df_tsne["tsne_2"]) + 1))

plt.legend(bbox_to_anchor=(1.13, 1.00), loc=loc, borderaxespad=0.0, fontsize=20, markerscale=2)
plt.axis("off")
plt.savefig(f"images/paper/Figure6_anomaly.png", bbox_inches='tight', dpi=dpi)

In [ ]:
df_origin = df_tsne[df_tsne["labels_origin"].notna()]
df_origin = df_origin.sort_values("labels_origin")

plt.figure(figsize=(16, 16))
sns.scatterplot(x='tsne_1', y='tsne_2', 
                hue='labels_origin', 
                style="labels_origin",
                data=df_origin, 
                s=50, 
                palette=colors[:2 * len(combination)],
                markers=markers[:2 * len(combination)]
               )
plt.xlim((np.min(df_tsne["tsne_1"]) - 1, np.max(df_tsne["tsne_1"]) + 1))
plt.ylim((np.min(df_tsne["tsne_2"]) - 1, np.max(df_tsne["tsne_2"]) + 1))

plt.legend(bbox_to_anchor=(1.13, 1.00), loc=loc, borderaxespad=0.0, fontsize=20, markerscale=2)
plt.axis("off")
plt.savefig(f"images/paper/Figure6_origin.png", bbox_inches='tight', dpi=dpi)

In [ ]:
df_risk = df_tsne[df_tsne["labels_risk"].notna()]
df_risk = df_risk.sort_values("labels_risk")

plt.figure(figsize=(16, 16))
sns.scatterplot(x='tsne_1', y='tsne_2', 
                hue='labels_risk', 
                style="labels_risk",
                data=df_risk, 
                s=50, 
                palette=colors[:2 * len(combination)],
                markers=markers[:2 * len(combination)]
               )
plt.xlim((np.min(df_tsne["tsne_1"]) - 1, np.max(df_tsne["tsne_1"]) + 1))
plt.ylim((np.min(df_tsne["tsne_2"]) - 1, np.max(df_tsne["tsne_2"]) + 1))

plt.legend(bbox_to_anchor=(1.13, 1.00), loc=loc, borderaxespad=0.0, fontsize=20, markerscale=2)
plt.axis("off")
plt.savefig(f"images/paper/Figure6_risk.png", bbox_inches='tight', dpi=dpi)

## Supplementary Table 1

In [ ]:
# Create demographics table for all datasets
dataset = pd.read_excel("demographics/raw_demographics_combined_images.xlsx")
columns = ["Age", "Sex", "BMI", "BMI class", "x", "y", "z", "Manufacturer", "Manufacturer Model Name", "Peak voltage (kVp)",
       "Exposure Time (s)", "Tube Current (mA)", "Exposure (mAs)"]
categorical = ["Sex", "BMI class", "Manufacturer", "Manufacturer Model Name", "Peak voltage (kVp)"]
non_normal = ["Age", "BMI", "x", "y", "z", "Exposure Time (s)", "Tube Current (mA)", "Exposure (mAs)"]
table_all = TableOne(dataset, columns=columns, categorical=categorical, groupby="Dataset", nonnormal=non_normal, pval=False, missing=False, overall=False)
table_all.to_excel(f"images/paper/SupplementaryTable1.xlsx")

## Supplementary Table 2

In [ ]:
# Create demographics table for all patient datasets
dataset = pd.read_excel("demographics/raw_demographics_combined_patients.xlsx")
columns = ["Age", "Sex", "BMI", "BMI class"]
categorical = ["Sex", "BMI class"]
non_normal = ["Age", "BMI"]
table_all = TableOne(dataset, columns=columns, categorical=categorical, groupby="Dataset", nonnormal=non_normal, pval=False, missing=False, overall=False)
table_all.to_excel(f"images/paper/SupplementaryTable2.xlsx")

## Supplemetnary Figure 5
Single models and ensemble ROC curves for all tasks and test datasets (as in Figure 3), only for male patients.

In [ ]:
gender = "m"
strategy = "train"
for task in tasks:
    df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
    df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))
    
    for df, set_name, color in zip([df_test_internal, df_test_external], ["test_internal", "test_external"], ["red", "blue"]):
        # PLot ROC curves for single models
        roc_curves = []
        for i in [0, 1, 2, 3, 4]:
            roc = compute_roc(X=df[f"model_{i}"], y=df["label"], pos_label=1)
            roc_curves.append(roc)
            plot_roc(roc, label=f"Model {i + 1}", color=color_palette[i])
        plt.title(add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - 5-Folds", gender, True))
        plt.savefig(f"images/paper/SupplementaryFigure5_{task}_{set_name}_singleModels.png", dpi=dpi)
        plt_show()
        
        # Plot ROC curve for the ensemble model
        plot_roc_bootstrap(X=df["ensemble"], y=df["label"], pos_label=1, label=f"{titles_dict[set_name]}",
                       color=color, n_bootstrap=n_boot, show_boots=False)
        plt.title(add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - Ensemble", gender, True)) 
        plt.suptitle("")     # Clear the subtitle
        plt.savefig(f"images/paper/SupplementaryFigure5_{task}_{set_name}_ensemble.png", dpi=dpi)
        plt_show()

## Supplementary Figure 6
Single models and ensemble ROC curves for all tasks and test datasets (as in Figure 3), only for female patients.

In [ ]:
gender = "f"
strategy = "train"
for task in tasks:
    df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
    df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))
    
    for df, set_name, color in zip([df_test_internal, df_test_external], ["test_internal", "test_external"], ["red", "blue"]):
        # PLot ROC curves for single models
        roc_curves = []
        for i in [0, 1, 2, 3, 4]:
            roc = compute_roc(X=df[f"model_{i}"], y=df["label"], pos_label=1)
            roc_curves.append(roc)
            plot_roc(roc, label=f"Model {i + 1}", color=color_palette[i])
        plt.title(add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - 5-Folds", gender, True))
        plt.savefig(f"images/paper/SupplementaryFigure6_{task}_{set_name}_singleModels.png", dpi=dpi)
        plt_show()
        
        # Plot ROC curve for the ensemble model
        plot_roc_bootstrap(X=df["ensemble"], y=df["label"], pos_label=1, label=f"{titles_dict[set_name]}",
                       color=color, n_bootstrap=n_boot, show_boots=False)
        plt.title(add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - Ensemble", gender, True)) 
        plt.suptitle("")     # Clear the subtitle
        plt.savefig(f"images/paper/SupplementaryFigure6_{task}_{set_name}_ensemble.png", dpi=dpi)
        plt_show()

## Supplementary Figure 7
Mean ROC curves across models for all tasks and test datasets.

In [ ]:
gender = ""
strategy = "train"
for task in tasks:
    df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
    df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))
    
    for df, set_name, color in zip([df_test_internal, df_test_external], ["test_internal", "test_external"], ["red", "blue"]):
        roc_curves = []
        for i in range(5):
            roc = compute_roc(X=df_test_external[f"model_{i}"], y=df_test_external["label"], pos_label=1)
            roc_curves.append(roc)
        # PLot the mean ROC curves across single models
        plot_mean_roc(roc_curves, show_ci=False, show_ti=False, show_all=True, color=color)
        plt.title(add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - Mean ROC", gender, True))
        plt.savefig(f"images/paper/SupplementaryFigure7_{task}_{set_name}.png", dpi=dpi)
        plt_show()

## Supplementary Figure 8
Same as Supplementary Figure 7, only for male patients

In [ ]:
gender = "m"
strategy = "train"
for task in tasks:
    df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
    df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))
    
    for df, set_name, color in zip([df_test_internal, df_test_external], ["test_internal", "test_external"], ["red", "blue"]):
        roc_curves = []
        for i in range(5):
            roc = compute_roc(X=df_test_external[f"model_{i}"], y=df_test_external["label"], pos_label=1)
            roc_curves.append(roc)
        # PLot the mean ROC curves across single models
        plot_mean_roc(roc_curves, show_ci=False, show_ti=False, show_all=True, color=color)
        plt.title(add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - Mean ROC", gender, True))
        plt.savefig(f"images/paper/SupplementaryFigure8_{task}_{set_name}.png", dpi=dpi)
        plt_show()

## Supplementary Figure 9
Same as Supplementary Figure 7, only for female patients

In [ ]:
gender = "f"
strategy = "train"
for task in tasks:
    input_path = join("results", task ,"train")
    df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
    df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))
    
    for df, set_name, color in zip([df_test_internal, df_test_external], ["test_internal", "test_external"], ["red", "blue"]):
        roc_curves = []
        for i in range(5):
            roc = compute_roc(X=df_test_external[f"model_{i}"], y=df_test_external["label"], pos_label=1)
            roc_curves.append(roc)
        # PLot the mean ROC curves across single models
        plot_mean_roc(roc_curves, show_ci=False, show_ti=False, show_all=True, color=color)
        plt.title(add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - Mean ROC", gender, True))
        plt.savefig(f"images/paper/SupplementaryFigure9_{task}_{set_name}.png", dpi=dpi)
        plt_show()

## Supplementary Figure 10
Confusion matrices for anomaly detection task at different classification thresholds, for internal and external testing datasets.

In [ ]:
gender = ""
strategy = "train"
task = "anomaly_detection"
input_path = join("results", task ,"train")
df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))

for df, set_name in zip([df_test_internal, df_test_external], ["test_internal", "test_external"]):
    for threshold in thresholds:
        title = add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - {threshold_title}: " + str(threshold), gender, True)
        plot_confusion_matrix(predictions=df["ensemble"] > threshold, labels=df["label"].astype(int),
                              output_path=f"images/paper/SupplementaryFigure10_{set_name}_{threshold}.png",
                              title=title, show=show, dpi=dpi, title_size=15)
        plt_show()

## Supplementary Figure 11
Same as Supplementary Figure 10 but only for male patients.

In [ ]:
gender = "m"
strategy = "train"
df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))

for df, set_name in zip([df_test_internal, df_test_external], ["test_internal", "test_external"]):
    for threshold in thresholds:
        title = add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - {threshold_title}: " + str(threshold), gender, True)
        plot_confusion_matrix(predictions=df["ensemble"] > threshold, labels=df["label"].astype(int),
                              output_path=f"images/paper/SupplementaryFigure11_{set_name}_{threshold}.png",
                              title=title, show=show, dpi=dpi, title_size=15)
        plt_show()

## Supplementary Figure 12
Same as Supplementary Figure 10 but only for female patients.

In [ ]:
gender = "f"
strategy = "train"
df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))

for df, set_name in zip([df_test_internal, df_test_external], ["test_internal", "test_external"]):
    for threshold in thresholds:
        title = add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - {threshold_title}: " + str(threshold), gender, True)
        plot_confusion_matrix(predictions=df["ensemble"] > threshold, labels=df["label"].astype(int),
                              output_path=f"images/paper/SupplementaryFigure12_{set_name}_{threshold}.png",
                              title=title, show=show, dpi=dpi, title_size=15)
        plt_show()

## Supplementary Figure 13
Confusion matrices for origin classification task at different classification thresholds, for internal and external testing datasets.

In [ ]:
task = "origin_classification"
input_path = join("results", task ,"train")
gender = ""
strategy = "train"
df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))

for df, set_name in zip([df_test_internal, df_test_external], ["test_internal", "test_external"]):
    for threshold in thresholds:
        title = add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - {threshold_title}: " + str(threshold), gender, True)
        plot_confusion_matrix(predictions=df["ensemble"] > threshold, labels=df["label"].astype(int),
                              output_path=f"images/paper/SupplementaryFigure13_{set_name}_{threshold}.png",
                              title=title, show=show, dpi=dpi)
        plt_show()

## Supplementary Figure 14
Same as Supplementary Figure 13 but only for male patients.

In [ ]:
gender = "m"
strategy = "train"
df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))

for df, set_name in zip([df_test_internal, df_test_external], ["test_internal", "test_external"]):
    for threshold in thresholds:
        title = add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - {threshold_title}: " + str(threshold), gender, True)
        plot_confusion_matrix(predictions=df["ensemble"] > threshold, labels=df["label"].astype(int),
                              output_path=f"images/paper/SupplementaryFigure14_{set_name}_{threshold}.png",
                              title=title, show=show, dpi=dpi)
        plt_show()

## Supplementary Figure 15
Same as Supplementary Figure 13 but only for female patients.

In [ ]:
gender = "f"
strategy = "train"
df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))

for df, set_name in zip([df_test_internal, df_test_external], ["test_internal", "test_external"]):
    for threshold in thresholds:
        title = add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - {threshold_title}: " + str(threshold), gender, True)
        plot_confusion_matrix(predictions=df["ensemble"] > threshold, labels=df["label"].astype(int),
                              output_path=f"images/paper/SupplementaryFigure15_{set_name}_{threshold}.png",
                              title=title, show=show, dpi=dpi)
        plt_show()

## Supplementary Figure 16
Confusion matrices for **origin classification** task at different classification thresholds, for internal and external testing datasets.

In [ ]:
task = "risk_classification"
input_path = join("results", task ,"train")
gender = ""
strategy = "train"
df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))

for df, set_name in zip([df_test_internal, df_test_external], ["test_internal", "test_external"]):
    for threshold in thresholds:
        title = add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - {threshold_title}: " + str(threshold), gender, True)
        plot_confusion_matrix(predictions=df["ensemble"] > threshold, labels=df["label"].astype(int),
                              output_path=f"images/paper/SupplementaryFigure16_{set_name}_{threshold}.png",
                              title=title, show=show, dpi=dpi)
        plt_show()

## Supplementary Figure 17
Same as Supplementary Figure 16 but only for male patients.

In [ ]:
gender = "m"
strategy = "train"
df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))

for df, set_name in zip([df_test_internal, df_test_external], ["test_internal", "test_external"]):
    for threshold in thresholds:
        title = add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - {threshold_title}: " + str(threshold), gender, True)
        plot_confusion_matrix(predictions=df["ensemble"] > threshold, labels=df["label"].astype(int),
                              output_path=f"images/paper/SupplementaryFigure17_{set_name}_{threshold}.png",
                              title=title, show=show, dpi=dpi)
        plt_show()

## Supplementary Figure 18
Same as Supplementary Figure 16 but only for female patients.

In [ ]:
gender = "f"
strategy = "train"
df_test_internal = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_internal_{strategy}.xlsx"), gender))
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))

for df, set_name in zip([df_test_internal, df_test_external], ["test_internal", "test_external"]):
    for threshold in thresholds:
        title = add_gender(f"{task.replace("_", " ").capitalize()} - {titles_dict[set_name]} - {threshold_title}: " + str(threshold), gender, True)
        plot_confusion_matrix(predictions=df["ensemble"] > threshold, labels=df["label"].astype(int),
                              output_path=f"images/paper/SupplementaryFigure18_{set_name}_{threshold}.png",
                              title=title, show=show, dpi=dpi)
        plt_show()

## Supplementary Figure 19
Single models, mean and ensemble ROC curves for anomaly detection task and external testing dataset, with models trained on the entire internal dataset (**strategy 2**).

In [ ]:
task = "anomaly_detection"
input_path = join("results", task ,"strategy2")
gender = ""
set_name = "test_external"
color = "blue"

In [ ]:
fig_base_name = "SupplementaryFigure19"
strategy = "strategy2"
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))
# PLot ROC curves for single models
roc_curves = []
for i in [0, 1, 2, 3, 4]:
    roc = compute_roc(X=df_test_external[f"model_{i}"], y=df_test_external["label"], pos_label=1)
    roc_curves.append(roc)
    plot_roc(roc, label=f"Model {i + 1}", color=color_palette[i])
plt.title(add_gender(f"{titles_dict[set_name]} - 5-Folds", gender, True))
plt.savefig(f"images/paper/{fig_base_name}_singleModels.png", dpi=dpi)
plt_show()

# Plot ROC curve for the ensemble model
plot_roc_bootstrap(X=df_test_external["ensemble"], y=df_test_external["label"], pos_label=1, label=f"{titles_dict[set_name]}",
                   color=color, n_bootstrap=n_boot, show_boots=False)
plt.title(add_gender(f"{titles_dict[set_name]} - Ensemble", gender, True)) 
plt.suptitle("")     # Clear the subtitle
plt.savefig(f"images/paper/{fig_base_name}_ensemble.png", dpi=dpi)
plt_show()

# PLot the mean ROC curves across single models
plot_mean_roc(roc_curves, show_ci=False, show_ti=False, show_all=True, color=color)
plt.title(add_gender(f"{titles_dict[set_name]} - Mean ROC", gender, True))
plt.savefig(f"images/paper/{fig_base_name}_meanROC.png", dpi=dpi)
plt_show()

## Supplementary Figure 20
Same as Supplementary Figure 20, but only for male patients.

In [ ]:
gender = "m"
strategy = "strategy2"
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))
fig_base_name = "SupplementaryFigure20"
# PLot ROC curves for single models
roc_curves = []
for i in [0, 1, 2, 3, 4]:
    roc = compute_roc(X=df_test_external[f"model_{i}"], y=df_test_external["label"], pos_label=1)
    roc_curves.append(roc)
    plot_roc(roc, label=f"Model {i + 1}", color=color_palette[i])
plt.title(add_gender(f"{titles_dict[set_name]} - 5-Folds", gender, True))
plt.savefig(f"images/paper/{fig_base_name}_singleModels.png", dpi=dpi)
plt_show()

# Plot ROC curve for the ensemble model
plot_roc_bootstrap(X=df_test_external["ensemble"], y=df_test_external["label"], pos_label=1, label=f"{titles_dict[set_name]}",
                   color=color, n_bootstrap=n_boot, show_boots=False)
plt.title(add_gender(f"{titles_dict[set_name]} - Ensemble", gender, True)) 
plt.suptitle("")     # Clear the subtitle
plt.savefig(f"images/paper/{fig_base_name}_ensemble.png", dpi=dpi)
plt_show()

# PLot the mean ROC curves across single models
plot_mean_roc(roc_curves, show_ci=False, show_ti=False, show_all=True, color=color)
plt.title(add_gender(f"{titles_dict[set_name]} - Mean ROC", gender, True))
plt.savefig(f"images/paper/{fig_base_name}_meanROC.png", dpi=dpi)
plt_show()

## Supplementary Figure 21
Same as Supplementary Figure 21, but only for female patients.

In [ ]:
gender = "f"
strategy = "strategy2"
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))
fig_base_name = "SupplementaryFigure21"
# PLot ROC curves for single models
roc_curves = []
for i in [0, 1, 2, 3, 4]:
    roc = compute_roc(X=df_test_external[f"model_{i}"], y=df_test_external["label"], pos_label=1)
    roc_curves.append(roc)
    plot_roc(roc, label=f"Model {i + 1}", color=color_palette[i])
plt.title(add_gender(f"{titles_dict[set_name]} - 5-Folds", gender, True))
plt.savefig(f"images/paper/{fig_base_name}_singleModels.png", dpi=dpi)
plt_show()

# Plot ROC curve for the ensemble model
plot_roc_bootstrap(X=df_test_external["ensemble"], y=df_test_external["label"], pos_label=1, label=f"{titles_dict[set_name]}",
                   color=color, n_bootstrap=n_boot, show_boots=False)
plt.title(add_gender(f"{titles_dict[set_name]} - Ensemble", gender, True)) 
plt.suptitle("")     # Clear the subtitle
plt.savefig(f"images/paper/{fig_base_name}_ensemble.png", dpi=dpi)
plt_show()

# PLot the mean ROC curves across single models
plot_mean_roc(roc_curves, show_ci=False, show_ti=False, show_all=True, color=color)
plt.title(add_gender(f"{titles_dict[set_name]} - Mean ROC", gender, True))
plt.savefig(f"images/paper/{fig_base_name}_meanROC.png", dpi=dpi)
plt_show()

## Supplementary Figure 22
Confusion matrix at different thresholds for anomaly detection task and external testing dataset, with models trained on the entire internal dataset (strategy 2).

In [ ]:
gender = ""
strategy = "strategy2"
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))
for threshold in thresholds:
    title = add_gender(f"{titles_dict[set_name]} - {threshold_title}: " + str(threshold), gender, True)
    plot_confusion_matrix(predictions=df_test_external["ensemble"] > threshold, labels=df_test_external["label"].astype(int),
                          output_path=f"images/paper/SupplementaryFigure22_{threshold}.png",
                          title=title, show=show, dpi=dpi)
    plt_show()

## Supplementary Figure 23
Same as Supplementary Figure 22, but only for male patients.

In [ ]:
gender = "m"
strategy = "strategy2"
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))
for threshold in thresholds:
    title = add_gender(f"{titles_dict[set_name]} - {threshold_title}: " + str(threshold), gender, True)
    plot_confusion_matrix(predictions=df_test_external["ensemble"] > threshold, labels=df_test_external["label"].astype(int),
                          output_path=f"images/paper/SupplementaryFigure23_{threshold}.png",
                          title=title, show=show, dpi=dpi)
    plt_show()

## Supplementary Figure 24
Same as Supplementary Figure 22, but only for female patients.

In [ ]:
gender = "f"
strategy = "strategy2"
df_test_external = pd.read_excel(add_gender(join(input_path, f"predictions_{task}_test_external_{strategy}.xlsx"), gender))
for threshold in thresholds:
    title = add_gender(f"{titles_dict[set_name]} - {threshold_title}: " + str(threshold), gender, True)
    plot_confusion_matrix(predictions=df_test_external["ensemble"] > threshold, labels=df_test_external["label"].astype(int),
                          output_path=f"images/paper/SupplementaryFigure24_{threshold}.png",
                          title=title, show=show, dpi=dpi)
    plt_show()